In [9]:
import sys

if len(sys.argv) > 1:
    number = int(sys.argv[1])
else:
    number = 5 * 10**6

print("Ejecutando con number = %d" % number)

ValueError: invalid literal for int() with base 10: '-f'

## Summing all the prime numbers below a given number

In [10]:
import time

# Simple code

def if_prime(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i**2 <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

def sum_primes(x):
    result = 0
    for i in range(x):
        result += if_prime(i)
    return result

number = 2_500_000
suma = 0
N = 3 # number of loops

start = time.time()
for i in range(N):
    suma = sum(map(if_prime, list(range(number))))
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes(number)
suma = sum_primes(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)


The prime sum below  2500000 is  219697708195  and the time taken is 4.738808234532674
The prime sum below  2500000 is  219697708195  and the time taken is 4.79 s ± 26.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


## Optimización con Numba secuencial.

In [11]:
import time
from numba import njit

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_primes_numba(x):
    result = 0
    for i in range(x):
        result += if_prime_numba(i)
    return result

number = 2_500_000
suma = 0
N = 3

sum_primes_numba(number)

start = time.time()
for i in range(N):
    suma = sum_primes_numba(number)
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_numba(number)
suma = sum_primes_numba(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

The prime sum below  2500000 is  219697708195  and the time taken is 0.182099978129069
The prime sum below  2500000 is  219697708195  and the time taken is 182 ms ± 21.6 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)


## Operación Multiprocessing con Pool

In [12]:
import time
from multiprocessing import Pool
from numba import njit

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit
def sum_range_numba(start, end):
    total = 0
    for i in range(start, end):
        total += if_prime_numba(i)
    return total

def sum_primes_mp(x, nproc):
    chunk = x // nproc
    ranges = [(i * chunk, (i + 1) * chunk) for i in range(nproc)]
    ranges[-1] = (ranges[-1][0], x)
    with Pool(nproc) as p:
        partial = p.starmap(sum_range_numba, ranges)
    return sum(partial)

def sum_primes_mp_4(x):
    return sum_primes_mp(x, 4)

number = 2_500_000
suma = 0
N = 3

start = time.time()
for i in range(N):
    suma = sum_primes_mp_4(number)
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_mp_4(number)
suma = sum_primes_mp_4(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

The prime sum below  2500000 is  219697708195  and the time taken is 0.22152201334635416
The prime sum below  2500000 is  219697708195  and the time taken is 220 ms ± 130 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)


## Operación Numba con prange.

In [13]:
import time
from numba import njit, prange

@njit
def if_prime_numba(x):
    if x <= 1:
        return 0
    elif x <= 3:
        return x
    elif x % 2 == 0 or x % 3 == 0:
        return 0
    i = 5
    while i * i <= x:
        if x % i == 0 or x % (i + 2) == 0:
            return 0
        i += 6
    return x

@njit(parallel=True)
def sum_primes_prange(x):
    total = 0
    for i in prange(x):
        total += if_prime_numba(i)
    return total

number = 2_500_000
suma = 0
N = 3

sum_primes_prange(number)

start = time.time()
for i in range(N):
    suma = sum_primes_prange(number)
stop = time.time()
tiempo = (stop - start) / N

print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

tiempo = %timeit -r 2 -o -q sum_primes_prange(number)
suma = sum_primes_prange(number)
print("The prime sum below ", number, "is ", suma, " and the time taken is", tiempo)

The prime sum below  2500000 is  219697708195  and the time taken is 0.034474690755208336
The prime sum below  2500000 is  219697708195  and the time taken is 32.5 ms ± 911 ns per loop (mean ± std. dev. of 2 runs, 10 loops each)


## Resultados de los analisis

Al comparar los resultados obtenidos para las unidades descritas en el ejercicio, se observa un comportamiento claramente distinto en función del enfoque utilizado. En el caso del código original en Python, el aumento del número de elementos provoca un crecimiento prácticamente proporcional del tiempo de ejecución, manteniéndose el carácter claramente ineficiente del algoritmo. Para 10^7 elementos, el tiempo se incrementa de forma muy significativa respecto a 10^6, confirmando que este enfoque no escala adecuadamente y se ve completamente limitado por el coste de la interpretación.

En la versión secuencial optimizada con Numba (@njit), el incremento del tamaño del problema de 10^6 a 10^7 elementos se traduce en un aumento del tiempo de ejecución mucho más moderado. Aunque el tiempo crece de forma casi lineal con el número de elementos, la pendiente es mucho menor que en el código original, lo que demuestra que la compilación JIT permite aprovechar mejor la arquitectura subyacente y reducir drásticamente el overhead por iteración.

Por su parte, el enfoque basado en multiprocessing muestra una diferencia más acusada entre 10^6 y 10^7 elementos. Para tamaños pequeños como 10^6, el overhead de creación de procesos y comunicación penaliza notablemente el rendimiento, haciendo que la mejora frente a la versión secuencial sea limitada. Sin embargo, al aumentar el tamaño del problema hasta 10^7 elementos, la paralelización resulta más efectiva, aunque sin llegar a un escalado ideal.

Finalmente, Numba con prange es el enfoque que mejor gestiona el aumento del tamaño del problema. Al pasar de 10^6 a 10^7 elementos, el tiempo de ejecución aumenta de forma controlada y el paralelismo sigue siendo eficiente incluso al utilizar varios núcleos, permitiendo que el rendimiento escale correctamente con el tamaño del problema, convirtiéndose en la solución más robusta y eficiente para problemas de gran tamaño.

Ejecutando con 1000000 elementos

1 nucleo:

Ejecutando con number = 1000000

The prime sum below 2500000 is 219697708195 and the time taken is 8.28457244237264

The prime sum below 2500000 is 219697708195 and the time taken is 8.09 s ± 33.5 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.36196303367614746

The prime sum below 2500000 is 219697708195 and the time taken is 362 ms ± 9.14 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.3494456609090169

The prime sum below 2500000 is 219697708195 and the time taken is 347 ms ± 181 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.0176850159962972

The prime sum below 2500000 is 219697708195 and the time taken is 23.1 ms ± 152 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)

2 nucleos:

Ejecutando con number = 1000000

The prime sum below 2500000 is 219697708195 and the time taken is 8.408135175704956

The prime sum below 2500000 is 219697708195 and the time taken is 8.16 s ± 14.4 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.36193641026814777

The prime sum below 2500000 is 219697708195 and the time taken is 362 ms ± 17.5 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.35366344451904297

The prime sum below 2500000 is 219697708195 and the time taken is 353 ms ± 42.6 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.020668745040893555

The prime sum below 2500000 is 219697708195 and the time taken is 22.3 ms ± 652 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)

4 nucleos:

Ejecutando con number = 1000000

The prime sum below 2500000 is 219697708195 and the time taken is 8.401086012522379

The prime sum below 2500000 is 219697708195 and the time taken is 8.25 s ± 41.2 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.36207087834676105

The prime sum below 2500000 is 219697708195 and the time taken is 362 ms ± 1.03 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.35341986020406085

The prime sum below 2500000 is 219697708195 and the time taken is 354 ms ± 303 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.0232852300008138

The prime sum below 2500000 is 219697708195 and the time taken is 20.8 ms ± 1.92 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)

8 nucleos:

Ejecutando con number = 1000000

The prime sum below 2500000 is 219697708195 and the time taken is 8.268226305643717

The prime sum below 2500000 is 219697708195 and the time taken is 8.08 s ± 36.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.36194507280985516

The prime sum below 2500000 is 219697708195 and the time taken is 362 ms ± 13.3 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.3491150538126628

The prime sum below 2500000 is 219697708195 and the time taken is 349 ms ± 896 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.025081078211466473

The prime sum below 2500000 is 219697708195 and the time taken is 23.1 ms ± 257 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)
____________________________________________________________________________________________________________________________

Ejecutando con 10000000 elementos

1 nucleo:

Ejecutando con number = 10000000

The prime sum below 2500000 is 219697708195 and the time taken is 8.323460976282755

The prime sum below 2500000 is 219697708195 and the time taken is 8.12 s ± 30.9 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.36208224296569824

The prime sum below 2500000 is 219697708195 and the time taken is 362 ms ± 2.38 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.35628294944763184

The prime sum below 2500000 is 219697708195 and the time taken is 356 ms ± 227 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.023798545201619465

The prime sum below 2500000 is 219697708195 and the time taken is 20.5 ms ± 2.68 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)

2 nucleos:

Ejecutando con number = 10000000

The prime sum below 2500000 is 219697708195 and the time taken is 8.469431082407633

The prime sum below 2500000 is 219697708195 and the time taken is 8.1 s ± 45.1 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.36214129130045575

The prime sum below 2500000 is 219697708195 and the time taken is 362 ms ± 19.3 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.3488620122273763

The prime sum below 2500000 is 219697708195 and the time taken is 350 ms ± 569 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.025542418162027996

The prime sum below 2500000 is 219697708195 and the time taken is 22.1 ms ± 1.39 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)

4 nucleos:

Ejecutando con number = 10000000

The prime sum below 2500000 is 219697708195 and the time taken is 8.328731298446655

The prime sum below 2500000 is 219697708195 and the time taken is 8.14 s ± 45 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.36202168464660645

The prime sum below 2500000 is 219697708195 and the time taken is 362 ms ± 27 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.3517421881357829

The prime sum below 2500000 is 219697708195 and the time taken is 350 ms ± 110 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.026488463083902996

The prime sum below 2500000 is 219697708195 and the time taken is 22.7 ms ± 202 μs per loop (mean ± std. dev. of 2 runs, 10 loops each)

8 nucleos:

Ejecutando con number = 10000000

The prime sum below 2500000 is 219697708195 and the time taken is 8.32636555035909

The prime sum below 2500000 is 219697708195 and the time taken is 8.11 s ± 35.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.36211609840393066

The prime sum below 2500000 is 219697708195 and the time taken is 362 ms ± 31.6 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.3500574429829915

The prime sum below 2500000 is 219697708195 and the time taken is 352 ms ± 825 μs per loop (mean ± std. dev. of 2 runs, 1 loop each)

The prime sum below 2500000 is 219697708195 and the time taken is 0.021997451782226562

The prime sum below 2500000 is 219697708195 and the time taken is 21.3 ms ± 1.4 ms per loop (mean ± std. dev. of 2 runs, 10 loops each)